<a href="https://colab.research.google.com/github/rosie-xue/NLP/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 长短期记忆网络（LSTM）

长短期记忆网络（LSTM）是循环神经网络（RNN）的一种特殊类型，它能够学习长期依赖关系。

LSTM的设计旨在避免长期依赖问题，其默认行为是长时间记住信息。

![LSTM 单元图](lstm.png)

## LSTM 单元的组成部分

1. **遗忘门（Forget Gate）**：决定哪些信息将被丢弃或保留。它通过一个激活函数（如sigmoid）接收上一个隐藏状态 `h_{t-1}` 和当前输入 `x_t`，为单元状态 `C_{t-1}` 中的每个数输出一个在0到1之间的值。1代表“完全保留”，而0代表“完全丢弃”。

2. **输入门（Input Gate）**：决定哪些新的信息被储存在单元状态中。它包含两部分：一个sigmoid层决定哪些值将要更新，和一个tanh层创造一个新的候选值向量 `C̃_t`，这可能会被添加到单元状态中。

3. **单元状态（Cell State）**：LSTM单元的“记忆”部分，它在整个链上运行，只有轻微的线性交互，信息流动较为容易。它通过遗忘门来丢弃不需要的信息，并通过输入门来添加新的信息，形成新的单元状态 `C_t`。

4. **输出门（Output Gate）**：决定下一个隐藏状态 `h_t` 的内容。隐藏状态包含了先前输入的信息。输出门先通过一个sigmoid层决定哪一部分的单元状态将被输出，然后将单元状态通过tanh处理（得到一个在-1到1之间的值）并乘以sigmoid层的输出，以此决定最终输出什么。

LSTM有能力通过所谓的门结构精细调节地添加或移除信息到单元状态中。门是一种选择性地让信息通过的方式。它们由一个sigmoid神经网络层和逐点乘法操作组成。

LSTM具有添加或移除单元状态信息的能力，这一过程通过门结构仔细调节。

## LSTM 的应用

LSTM非常适合用于对时间序列数据进行分类、处理和预测，因为时间序列中重要事件之间可能存在未知持续时间的延迟。

LSTM被开发用来解决在训练传统RNN时可能遇到的梯度消失问题。


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import random
import os
from tqdm import tqdm

# 设置随机种子以确保结果可复现
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# 检查CUDA是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 定义一个继承自torch.utils.data.Dataset的类，用于加载和处理古诗数据集
class PoemDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, sequence_length=50):
        # file_path: 数据文件路径
        # sequence_length: 每个输入序列的长度
        self.sequence_length = sequence_length
        self.poems = open(file_path, 'r', encoding='utf-8').read().split('\n')
        self.char2idx = {}
        self.idx2char = {}
        self.data = []

        self.preprocess()

    def preprocess(self):
        text = ''.join(self.poems)
        chars = sorted(list(set(text)))
        # 编码词表
        self.char2idx = {ch: i for i, ch in enumerate(chars)}
        # 解码词表
        self.idx2char = {i: ch for i, ch in enumerate(chars)}
        print('词表长度：%d'%len(self.char2idx))

        for poem in self.poems:
            encoded_poem = [self.char2idx[ch] for ch in poem]
            for i in range(0, len(encoded_poem) - self.sequence_length):
                self.data.append(encoded_poem[i:i + self.sequence_length + 1])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # 根据索引获取一个样本，并将其转换为Tensor
        # sequence: 输入序列
        # target: 目标序列（即下一个字符的序列）
        sequence = torch.tensor(self.data[index][:-1], dtype=torch.long).to(device)
        target = torch.tensor(self.data[index][1:], dtype=torch.long).to(device)
        return sequence, target

In [ ]:
# 定义一个继承自nn.Module的LSTM模型类
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128, num_layers=1):
        super(LSTMModel, self).__init__()
        # 定义隐藏层的维度
        self.hidden_dim = hidden_dim
        # 定义一个嵌入层，将输入的索引转换为固定维度的嵌入向量
        self.embed = nn.Embedding(vocab_size, embed_dim)
        # 定义LSTM层，用于处理序列数据，参数包括输入的嵌入维度、隐藏层维度和LSTM层数
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers, batch_first=True)
        # 定义一个全连接层，将LSTM的输出转换为最终的预测结果，输出维度等于词汇表大小
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden):
        embeds = self.embed(x)
        # 通过LSTM层处理嵌入向量序列，得到序列的输出和新的隐藏状态
        lstm_out, hidden = self.lstm(embeds, hidden)
        # 将LSTM的输出通过全连接层，得到每个位置的词汇预测
        out = self.fc(lstm_out)
        return out, hidden

    def init_hidden(self, batch_size):
        # 初始化隐藏状态和细胞状态，全部为0，形状为[层数*方向数, 批量大小, 隐藏层维度]
        # 这里的.to(device)是为了确保张量在正确的设备上（如CPU或GPU），需要根据实际环境进行设置
        return (torch.zeros(1, batch_size, self.hidden_dim).to(device),
                torch.zeros(1, batch_size, self.hidden_dim).to(device))


In [ ]:
# 测试代码
def predict(model, start_text='天', prediction_len=100):
    hidden = model.init_hidden(1)
    start_input = torch.tensor([[dataset.char2idx[s] for s in start_text]], dtype=torch.long).to(device)
    predicted_text = start_text

    for _ in range(prediction_len):
        output, hidden = model(start_input, hidden)
        predicted_index = torch.argmax(output, dim=2)[:, -1]
        predicted_text += dataset.idx2char[predicted_index.item()]
        start_input = predicted_index.unsqueeze(0)

    print(predicted_text)


In [ ]:
# 数据集和数据加载器
dataset = PoemDataset('poems.txt')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# 模型实例化
vocab_size = len(dataset.char2idx)
model = LSTMModel(vocab_size).to(device)
loss_function = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

# 训练模型
for epoch in range(10):  # 总共训练10轮
    hidden = model.init_hidden(32)
    for batch, (seq, targets) in enumerate(tqdm(data_loader)):
        hidden = tuple([each.data for each in hidden])
        optimizer.zero_grad()
        output, hidden = model(seq, hidden)
        loss = loss_function(output.transpose(1, 2), targets)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')
    predict(model, '春江花月夜，', prediction_len=6)

词表长度：7647


100%|██████████| 43987/43987 [07:22<00:00, 99.45it/s] 


Epoch: 1, Loss: 4.663071155548096
春江花月夜，一片云帆，月


100%|██████████| 43987/43987 [07:21<00:00, 99.69it/s] 


Epoch: 2, Loss: 4.636305809020996
春江花月夜，风吹露华。一


...

Epoch: 8, Loss: 4.411494255065918
春江花月夜，不见江南袚禊


100%|██████████| 43987/43987 [07:21<00:00, 99.56it/s]


Epoch: 9, Loss: 4.4548444747924805
春江花月夜，不见君王说。


100%|██████████| 43987/43987 [07:23<00:00, 99.26it/s] 

Epoch: 10, Loss: 4.540146350860596
春江花月夜，不知何处，春


# QUESTIONS:
## 1.LSTM的工作原理是什么?
LSTM（长短期记忆）的工作原理基于其独特的内部结构，使其能够有效地处理时序数据并捕捉长期依赖关系。以下是LSTM工作原理的详细解释：

1. **细胞状态**：LSTM的核心是细胞状态，它类似于一个传送带，在整个链上运行并保存需要长期记忆的信息。这个细胞状态只有少量的线性交互，使得信息在上面流传时能够保持不变。
2. **门控结构**：LSTM通过门控结构来管理细胞状态中的信息。这些门控结构包括输入门、遗忘门和输出门。每个门都由一个sigmoid神经网络层和一个按位的乘法操作组成。

	**遗忘门**：遗忘门决定从细胞状态中丢弃哪些信息。它查看当前输入$x_t$和前一个隐藏状态$h_{t-1}$，然后输出一个0到1之间的值，这个值决定了细胞状态中哪些信息需要保留，哪些需要丢弃。\
	**输入门**：输入门决定哪些新信息将被存放到细胞状态中。它同样查看当前输入$x_t$和前一个隐藏状态$h_{t-1}$，然后输出两部分信息：一部分是用于更新细胞状态的候选值，另一部分是一个0到1之间的值，决定了新信息以何种程度更新到细胞状态中。\
	**输出门**：输出门决定细胞状态的哪些信息将被输出到隐藏状态$h_t$。它基于细胞状态和当前输入$x_t$，输出一个0到1之间的值，用于控制细胞状态的信息输出。

3. **信息更新**：通过遗忘门、输入门和输出门的协同作用，LSTM能够实现对细胞状态的更新和信息的输出。遗忘门决定了哪些旧信息被保留，输入门决定了哪些新信息被添加，而输出门则决定了最终输出的信息。

这种结构使得LSTM能够有效地处理长期依赖问题，因为它能够记住重要的信息并在需要时将其输出。因此，LSTM在自然语言处理、语音识别、时间序列预测等领域中得到了广泛应用。

总的来说，LSTM的工作原理是通过精心设计的门控结构和细胞状态来管理信息流动，实现对长期依赖关系的捕捉和处理。这使得LSTM成为处理时序数据的强大工具。
## 2.在处理长期依赖问题时，LSTM相较于传统的RNN有何优势?
LSTM（长短期记忆网络）在处理长期依赖问题时相较于传统的RNN（循环神经网络）具有显著优势，这主要归因于LSTM特有的结构设计。以下是LSTM相较于传统RNN在处理长期依赖问题时的几个主要优势：

1. **门控机制**：LSTM通过引入三个门（遗忘门、输入门和输出门）来控制信息的流动。这些门的作用是决定什么信息被允许进入记忆单元、保留在记忆单元，以及从记忆单元中输出。这种机制允许LSTM有效地过滤掉无关信息，只保留对当前任务有用的信息。

2. **遗忘门**：遗忘门允许LSTM丢弃掉不再重要的信息，这对于处理长期依赖问题至关重要。通过遗忘过时的信息，LSTM可以为新信息腾出空间，避免记忆单元被无关信息占据，从而更有效地学习长期依赖关系。

3. **长期记忆能力**：LSTM的设计使其能够维持长期的状态信息，这使得它在捕捉长期依赖方面特别有效。这与传统RNN相比，后者往往难以保持长期的状态信息，因为随着时间的推移，信息会逐渐消失（梯度消失问题）。

4. **梯度消失和爆炸问题的缓解**：传统的RNN在训练过程中常常会遇到梯度消失或梯度爆炸的问题，这使得网络难以学习长期依赖关系。LSTM通过其门控机制有效地缓解了这一问题，因为它允许梯度在较长的序列中稳定流动，而不会随时间显著减少或增加。

5. **灵活的信息流动控制**：LSTM的门控制机制提供了对信息流动的细粒度控制，这意味着模型可以学习什么时候更新记忆、什么时候保留记忆，以及什么时候将信息传递给下一个时间步。这种灵活性使LSTM在处理包含复杂依赖和不同时间尺度信息的序列数据时，比传统RNN表现更好。

总之，通过这些设计上的改进，LSTM相较于传统RNN在处理长时间序列和长期依赖问题时表现出了显著的优势，特别是在语言模型、时间序列预测和其他需要处理长期依赖信息的任务中。
## 3.为什么需要进行数据预处理，并且这个过程通常包括哪些步骤?
预处理过程包含了如下过程：原始文本预处理、向量化，构造可供模型训练使用的数据集
1. **文本读取与分割**：通过`open(file_path, 'r', encoding='utf-8').read().split('\n')`，代码读取指定文件路径的文本内容，然后按行分割存储每首诗。这个步骤是将原始文本数据加载到内存中。

2. **构建字符级词表**：通过`''.join(self.poems)`将所有诗句合并为一个长字符串，然后使用`set(text)`获取所有唯一字符，并通过`sorted(list(set(text)))`对这些字符进行排序。这一步骤生成了字符级别的词表。

3. **字符到索引的映射**：通过遍历排序后的唯一字符集，创建两个字典：`char2idx`（字符到索引的映射）和`idx2char`（索引到字符的映射）。这两个字典使得字符和数值索引之间可以相互转换，为文本的向量化打下基础。

4. **文本向量化**：对每首诗进行遍历，将诗中的每个字符使用`char2idx`映射为相应的索引，实现文本到数值的转换。这个步骤是将文本数据转换成模型可处理的数值形式。

5. **序列化处理**：通过内循环`for i in range(0, len(encoded_poem) - self.sequence_length)`，从每首向量化的诗中提取长度为`sequence_length`的序列，并且每个序列的目标是下一个字符。这样做既可以增加训练数据量，也使模型能够在给定当前序列的情况下预测下一个字符。

6. **构造训练样本**：每次提取的序列和其对应的下一个字符一起保存，形成一个个训练样本。这在`preprocess`方法的最后通过`self.data.append(encoded_poem[i:i + self.sequence_length + 1])`实现。

7. **数据加载和批处理**：通过实现`__len__`和`__getitem__`方法，`PoemDataset`类允许PyTorch的`DataLoader`访问和批量处理数据。`__getitem__`方法还将序列数据转换为PyTorch的`Tensor`类型，使其可以直接用于模型训练。

## 4.模型超参数对训练结果有什么影响? 尝试调整不同的超参数，比如embed_dim、hidden_dim、num_layers.观察并报告它们对模型性能的影响。
1. **embed_dim（嵌入维度）**: 指将输入数据（如单词或字符）转换为固定长度向量的维度。在NLP任务中，较大的嵌入维度可以捕获更多的语义信息。增加嵌入维度可以提高模型捕获输入数据细微差别的能力，有助于模型学习更加丰富的表示。然而，太大的嵌入维度可能导致过拟合，特别是在数据量较小的情况下，同时也会增加模型的参数数量和计算负担。
2. **hidden_dim（隐藏层维度）**：指LSTM单元中隐藏层的维度。它决定了模型在每个时间步可以保留的信息量。较大的隐藏层维度可以提供更强的学习能力，允许模型捕获更复杂的模式和长期依赖。但同样，过大的隐藏层维度会增加模型的复杂度，导致过拟合和计算成本上升。
3. **num_layers（层数）**:指LSTM模型中堆叠的LSTM层的数量。多层LSTM可以增强模型的表示能力。增加层数通常可以提升模型性能，使其能够学习更复杂的特征和层次结构。但是，层数过多可能导致难以训练（如梯度消失或梯度爆炸问题），并且会显著增加训练时间和模型参数数量。此外，过多的层数有时也会导致过拟合，特别是当训练数据不足以支持极其复杂模型时。
